In [ ]:
import os
import sys
import numpy as np
from pathlib import Path
repo_root = Path.cwd().parent
sys.path.append(str(repo_root))

In [ ]:
from src.fluence_utils import estimate_so2_from_dot, query_bkg_mua_for_pa, fit_bkg_mus_for_pa, compute_phi_homogeneous, compute_phi_heterogeneous, compute_phi_and_grad
lambda_list_dot = [730   , 785   , 808   , 830]
mua_list_dot    = [0.032 , 0.038 , 0.037 , 0.041]
mus_list_dot    = [7.860 , 7.489 , 7.155 , 7.830]
decomp_dot      = estimate_so2_from_dot(lambda_list_dot , mua_list_dot , verbose=True)
lambda_list_pat = [750   , 780   , 800   , 830]
bkg_mua_pat = query_bkg_mua_for_pa(lambda_list_pat , decomp_dot['c_oxy'] , decomp_dot['c_deoxy'])
print("Background mua at PAT wavelengths are " , bkg_mua_pat['Background mua'] , " cm\u207B\u00B9")
bkg_mus_pat = fit_bkg_mus_for_pa(lambda_list_dot, mus_list_dot, lambda_list_pat)
print("Background mus at PAT wavelengths are " , bkg_mus_pat['Background mus'] , " cm\u207B\u00B9")

In [ ]:
mu_a  = bkg_mua_pat['Background mua'][0]
mu_sp = bkg_mus_pat['Background mus'][0]
L_z_cm , L_x_cm , L_y_cm = 4.0 , 4.0 , 2.0
voxel_size_cm = (0.025,0.025,0.025)
dx, dy, dz = voxel_size_cm
Nz , Ny , Nx = int(L_z_cm/dz)+1 , int(L_y_cm/dy)+1 , int(L_x_cm/dx)+1
mu_a_map  = mu_a * np.ones((Nz, Ny, Nx))
mu_sp_map = mu_sp * np.ones((Nz, Ny, Nx))
phi_out, info, grad = compute_phi_and_grad(mu_a_cm=mu_a_map,
                                           mu_sp_cm=mu_sp_map,
                                           voxel_size_cm = voxel_size_cm,
                                           fiber_sigma_cm=0.03,
                                           compute_grad = False,
                                           verbose = True,
                                           dtype = np.float32)

In [ ]:
z_coords = np.linspace(0.0, L_z_cm, Nz)
x_coords = np.linspace(-L_x_cm/2.0, L_x_cm/2.0, Nx)
y_coords = np.linspace(-L_y_cm/2.0, L_y_cm/2.0, Ny)

# create empty map (we will fill background later to meet average constraint)
mu_a_map = np.empty((Nz, Ny, Nx), dtype=float)
mu_a_map.fill(np.nan)  # mark background with nan initially
mu_s_map = np.empty((Nz, Ny, Nx), dtype=float)
mu_s_map.fill(np.nan)

# 1) set first two z-layers to 5*mu_a (indices 0 and 1)
top_layers_idx = [0]  # first two along z axis
mu_a_map[top_layers_idx, :, :] = 2.0 * mu_a
mu_s_map[top_layers_idx, :, :] = 5.0 * mu_sp

# 2) create sphere at (z,y,x)=(1.5,0,0) radius 0.75 -> set to 3*mu_a
z0, y0, x0 = 1.5, 0.0, 0.0
radius = 0.5

# build 3D mesh for distance calculation
Z = z_coords[:, None, None]   # shape (Nz,1,1)
Y = y_coords[None, :, None]   # shape (1,Ny,1)
X = x_coords[None, None, :]   # shape (1,1,Nx)

dist2 = (Z - z0)**2 + (Y - y0)**2 + (X - x0)**2
sphere_mask = dist2 <= radius**2

# set sphere voxels to 3*mu_a (do not overwrite existing specified values)
# In your grid the sphere does not overlap with first two layers (but we avoid overwriting anyway)
mu_a_map[sphere_mask & np.isnan(mu_a_map)] = 50.0 * mu_a
mu_s_map[sphere_mask & np.isnan(mu_a_map)] = 0.05 * mu_sp

# If you prefer sphere to override top-layers when overlapping, use:
# mu_a_map[sphere_mask] = 3.0 * mu_a

# 3) compute background value b so that volume-average = mu_a
N_total = mu_a_map.size
specified_mask = ~np.isnan(mu_a_map)
N_specified = np.count_nonzero(specified_mask)
N_background = N_total - N_specified

sum_specified = np.nansum(mu_a_map)  # sum over specified voxels

if N_background == 0:
    # All voxels are specified; check average and warn if it doesn't equal mu_a
    avg_spec = sum_specified / N_total
    if not np.isclose(avg_spec, mu_a, atol=1e-12):
        raise ValueError("All voxels are already specified but average != mu_a; cannot set background.")
    b = None
else:
    # desired total sum = mu_a * N_total
    b = (mu_a * N_total - sum_specified) / N_background
    d = (mu_sp * N_total - sum_specified) / N_background
    # optionally ensure b >= 0 (raise or clip), here we check:
    if b < 0:
        raise ValueError(f"Computed background value b={b:.3e} is negative. Check inputs.")
    if d < 0:
        raise ValueError(f"Computed background value d={d:.3e} is negative. Check inputs.")
    mu_a_map[np.isnan(mu_a_map)] = b
    mu_s_map[np.isnan(mu_s_map)] = d

phi_out, info, grad = compute_phi_and_grad(mu_a_cm=mu_a_map,
                                           mu_sp_cm=mu_s_map,
                                           voxel_size_cm = voxel_size_cm,
                                           fiber_sigma_cm=0.03,
                                           compute_grad = False,
                                           verbose = True,
                                           dtype = np.float32)

In [ ]:
# plot center z-x slice
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm
fig_tmp = phi_out[Ny//2, :, :].T
fig_tmp = np.where(fig_tmp > 0, fig_tmp, np.nan)

plt.figure(dpi=400)
im1 = plt.imshow(
    fig_tmp,
    norm=LogNorm(vmin=np.nanmax([1e-6, np.nanmin(fig_tmp)]), vmax=np.nanmax(fig_tmp)),
    extent=[-2.0, 2.0, 0.0, 4.0],
    origin='lower',
    aspect='auto',
    cmap='inferno'
)
plt.xlabel("x (cm)")
plt.ylabel("z (cm)")
plt.colorbar(im1, label="Φ (log scale)")
plt.gca().invert_yaxis()  # <- safer than plt.invert_yaxis()
plt.show()

In [ ]:
# plot y-z slice
fig_tmp = phi_out[:, Nx//2, :].T
fig_tmp = np.where(fig_tmp > 0, fig_tmp, np.nan)

plt.figure(dpi=400)
im1 = plt.imshow(
    fig_tmp,
    norm=LogNorm(vmin=np.nanmax([1e-6, np.nanmin(fig_tmp)]), vmax=np.nanmax(fig_tmp)),
    extent=[-1.0, 1.0, 0.0, 4.0],
    origin='lower',
    cmap='inferno'
)
plt.xlabel("y (cm)")
plt.ylabel("z (cm)")
plt.colorbar(im1, label="Φ (log scale)")
plt.gca().invert_yaxis()  # <- safer than plt.invert_yaxis()
plt.show()

In [ ]:
mu_a  = bkg_mua_pat['Background mua'][0]
mu_sp = bkg_mus_pat['Background mus'][0]
phi_homogeneous = compute_phi_homogeneous(mu_a=mu_a, mu_sp=mu_sp , dx=0.1, dy=0.1, dz=0.1, sigma_src = 0.03, verbose=True)

In [ ]:
mu_a_map  = mu_a * np.ones((Nz, Ny, Nx))
mu_sp_map = mu_sp * np.ones((Nz, Ny, Nx))
phi_heterogeneous, info = compute_phi_heterogeneous(mu_a_cm=mu_a_map, mu_sp_cm=mu_sp_map, fiber_sigma_cm=0.03, verbose=True)